# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [108]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

filename_features = "/content/features.txt"
filename_labels = "/content/activity_labels.txt"

filename_subtrain = "/content/subject_train.txt"
filename_xtrain = "/content/X_train.txt"
filename_ytrain = "/content/y_train.txt"

filename_subtest = "/content/subject_test.txt"
ffilename_xtest = "/content/X_test.txt"
filename_ytest = "/content/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'])
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features['nome_var'])
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

In [109]:
df1 = pd.concat([X_train, X_test])
df2 = pd.concat([y_train, y_test[0:1869]])

In [110]:
df1['cod_label'] = df2['cod_label']

In [111]:
df1.dropna(inplace=True)
X = df1.drop(columns='cod_label')
y = df1['cod_label']

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [113]:
%%time
clf = DecisionTreeClassifier(ccp_alpha=0.001)
clf.fit(X_train, y_train)

CPU times: user 6.15 s, sys: 2.29 ms, total: 6.15 s
Wall time: 7.12 s


DecisionTreeClassifier(ccp_alpha=0.001)

In [114]:
%%time
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

CPU times: user 25.6 ms, sys: 96 µs, total: 25.7 ms
Wall time: 26.5 ms


In [115]:
%%time
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Acurácia na base de treinamento:", train_accuracy)
print("Acurácia na base de teste:", test_accuracy)

Acurácia na base de treinamento: 0.967326464208243
Acurácia na base de teste: 0.9343817787418656
CPU times: user 4.9 ms, sys: 933 µs, total: 5.84 ms
Wall time: 6.26 ms


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [116]:
def train_pca(n_components):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    start_time = time.time()
    clf = DecisionTreeClassifier(ccp_alpha=0.001)
    clf.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    y_train_pred = clf.predict(X_train_pca)
    y_test_pred = clf.predict(X_test_pca)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    return train_accuracy, test_accuracy, training_time

In [117]:
%%time
train_accuracy, test_accuracy = train_pca(1)
print("Acurácia na base de treinamento:", train_accuracy)
print("Acurácia na base de teste:", test_accuracy)

ValueError: too many values to unpack (expected 2)

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [118]:
%%time
results = []
for n in [1, 2, 5, 10, 50]:
  train_accuracy, test_accuracy, training_time = train_pca(n)
  results.append({
      'n_components': n,
      'train_accuracy': train_accuracy,
      'test_accuracy': test_accuracy,
      'training_time': training_time
  })

CPU times: user 4.38 s, sys: 2.11 s, total: 6.48 s
Wall time: 3.74 s


In [119]:
for result in results:
    print(f"Componentes: {result['n_components']}")
    print(f"Acurácia na base de treinamento: {result['train_accuracy']:.4f}")
    print(f"Acurácia na base de teste: {result['test_accuracy']:.4f}")
    print(f"Tempo de treinamento: {result['training_time']:.4f} segundos")
    print("-" * 30)

Componentes: 1
Acurácia na base de treinamento: 0.4966
Acurácia na base de teste: 0.5076
Tempo de treinamento: 0.1066 segundos
------------------------------
Componentes: 2
Acurácia na base de treinamento: 0.6245
Acurácia na base de teste: 0.6242
Tempo de treinamento: 0.0815 segundos
------------------------------
Componentes: 5
Acurácia na base de treinamento: 0.8320
Acurácia na base de teste: 0.8292
Tempo de treinamento: 0.0793 segundos
------------------------------
Componentes: 10
Acurácia na base de treinamento: 0.8772
Acurácia na base de teste: 0.8427
Tempo de treinamento: 0.1682 segundos
------------------------------
Componentes: 50
Acurácia na base de treinamento: 0.9094
Acurácia na base de teste: 0.8595
Tempo de treinamento: 0.6374 segundos
------------------------------


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

In [120]:
# A acurácia aumentou conforme o número de componentes aumenta, porém o tempo de treinamento aumenta proporcionalmente